In [ ]:
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm
import IPython.display as display
import copy
import seaborn as sns

In [ ]:
def merge_csv_files(directory):
    # Get a list of all the csv files
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

    # Initialize an empty list to hold dataframes
    dfs = []

    # Loop through csv files, read each into a dataframe, and append to the list
    for file in csv_files:
        # Extract date from filename, assuming the date is in format 'traffic_flow_YYYY_MM_DD'
        date_str = file.split('.')[0].split('_')[-3:]  # This gives ['YYYY', 'MM', 'DD']
        date = datetime.strptime('_'.join(date_str), '%Y_%m_%d').date()

        df = pd.read_csv(os.path.join(directory, file))

        # Add date as a new column
        df['date'] = date.strftime('%m/%d/%y')

        dfs.append(df)

    # Concatenate all dataframes in the list into one dataframe
    merged_df = pd.concat(dfs, ignore_index=True).drop_duplicates()

    # Return the merged dataframe
    return merged_df

In [ ]:
traffic_flows = merge_csv_files(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]Traffic flow')
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'

# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()

In [ ]:
lsoa = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/london_LSOA/london_LSOA.shp'
road_network = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/[XH]road_network/road_network.shp'
inoutter = '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/lp-consultation-oct-2009-inner-outer-london-shp/lp-consultation-oct-2009-inner-outer-london.shp'
tube_line = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_lines.json'
tube_station = 'https://raw.githubusercontent.com/oobrien/vis/master/tubecreature/data/tfl_stations.json'

inoutter = gpd.read_file(inoutter)
inoutter.to_crs(epsg=27700, inplace=True)

# tube_station = gpd.read_file(tube_station)
# tube_station.to_crs(epsg=27700, inplace=True)
# tube_station = gpd.sjoin(tube_station, inoutter, op='within')

tube_line = gpd.read_file(tube_line)
tube_line.to_crs(epsg=27700, inplace=True)
tube_line = gpd.sjoin(tube_line, inoutter, op='within')

lsoa = gpd.read_file(lsoa, crs={'init': 'epsg:27700'})
road_network = gpd.read_file(road_network, crs={'init': 'epsg:27700'})

In [ ]:
# clean the traffic flow data
traffic_flows = traffic_flows.drop_duplicates(['toid', 'date'])
traffic_flows = traffic_flows.groupby(['toid', 'date']).agg(
    {'bus': 'sum', 'car': 'sum', 'cycle': 'sum', 'walks': 'sum', 'stationary': 'sum'}).reset_index()
traffic_flows['total'] = traffic_flows['bus'] + traffic_flows['car'] + traffic_flows['cycle'] + traffic_flows[
    'walks'] + traffic_flows['stationary']

In [ ]:
flows = pd.merge(
    road_network[['toid', 'roadclassi', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg'
                  ]],
    traffic_flows, left_on='toid', right_on='toid', how='right')

flows['classification'] = flows['roadclassi'].replace(
    {'Unknown': 'Local Road', 'Not Classified': 'Local Road', 'Unclassified': 'Local Road',
     'Classified Unnumbered': 'Local Road'})

flows.drop(columns=['roadclassi'], inplace=True)

stage_date = ['03/01/22', '02/22/22', '03/08/22']
flows = flows.loc[flows['date'].isin(stage_date)]

# label the regional level
flows = gpd.sjoin(flows, inoutter, how='inner', predicate='within')
flows = flows.drop(columns=['index_right', 'Source', 'Area_Ha', 'Shape_Leng', 'Shape_Area'])
flows.reset_index(drop=True, inplace=True)

# convert the dataframe
flows = pd.melt(flows,
                id_vars=['toid', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
                         'date', 'Boundary'], var_name='mode', value_name='flow')

flows = pd.pivot_table(flows,
                       index=['toid', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
                              'date', 'Boundary', 'mode'], columns='date', values='flow', aggfunc='first').reset_index()

flows.drop(columns=['date'], inplace=True)
flows = flows.groupby(
    ['toid', 'mode', 'classification', 'geometry', 'directiona', 'length', 'roadwidthm', 'elevationg',
     'Boundary'], as_index=False).agg(
    {'03/01/22': 'first', '02/22/22': 'first', '03/08/22': 'first'})

# calculate the impact and recovery flows for one strike
flows['impact_flow'] = flows['03/01/22'] - flows['02/22/22']
flows['recovery_flow'] = flows['03/08/22'] - flows['03/01/22']

All = flows.copy()



### Take city of  london as an example

In [ ]:
# load the city of london LA
LA = gpd.read_file(
    '/Users/zonghe/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Zonghe Ma/Raw data/London administrative boundaries/LA/LA .shp',
    crs={'init': 'epsg:27700'})
CoL = LA.loc[LA['LAD22NM'] == 'City of London']

# spatial clip of the flows data
flows = gpd.GeoDataFrame(flows, geometry='geometry')
CoL_flows = gpd.clip(flows, CoL)

flows = CoL_flows.copy().reset_index(drop=True)

flows.plot()
plt.axis('off')
plt.show()

In [ ]:
flows

## Graph

In [ ]:
flows = flows[flows['mode'] == 'total']
flows.reset_index(drop=True)

# Create an empty graph
graph = nx.Graph()

In [ ]:
# creat a blank graph
graph = nx.Graph()

# iterate over the rows of the flows DataFrame
for _, row in flows.iterrows():
    geometry = row['geometry']
    baseline_1 = row['02/22/22']
    strike_1 = row['03/01/22']
    recovery_1 = row['03/08/22']
    impact_flow = row['impact_flow']
    recovery_flow = row['recovery_flow']
    direction = row['directiona']

    # break the MultiLineString geometry into its constituent LineStrings
    if geometry.geom_type == 'MultiLineString':
        for line_string in geometry.geoms:  # iterate over each LineString
            from_node = line_string.coords[0]
            to_node = line_string.coords[-1]

            # Add nodes to the graph
            graph.add_node(from_node, pos=from_node)  # Use 'from_node' as the node position
            graph.add_node(to_node, pos=to_node)  # Use 'to_node' as the node position
            # Add edges to the graph based on the direction
            if direction == 'bothDirections':
                # If the road is bidirectional, flows are split equally in both directions
                graph.add_edge(from_node, to_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2,
                               recovery_1=recovery_1 / 2, impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                               toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
                graph.add_edge(to_node,
                               from_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2, recovery_1=recovery_1 / 2,
                               impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                               toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
            elif direction == 'inOppositeDirection':
                # If the road is in the opposite direction, flows are from the ending point to the starting point
                graph.add_edge(to_node,
                               from_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2, recovery_1=recovery_1 / 2,
                               impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                               toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
            elif direction == 'inDirection':
                # If the road is in the same direction, flows are from the starting point to the ending point
                graph.add_edge(from_node, to_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2,
                               recovery_1=recovery_1 / 2, impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                               toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
    else:
        from_node = geometry.coords[0]
        to_node = geometry.coords[-1]

        # Add nodes to the graph
        graph.add_node(from_node, pos=from_node)  # Use 'from_node' as the node position
        graph.add_node(to_node, pos=to_node)  # Use 'to_node' as the node position
        # Add edges to the graph based on the direction
        if direction == 'bothDirections':
            # If the road is bidirectional, flows are split equally in both directions
            graph.add_edge(from_node, to_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2,
                           recovery_1=recovery_1 / 2, impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                           toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
            graph.add_edge(to_node, from_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2,
                           recovery_1=recovery_1 / 2, impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                           toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
        elif direction == 'inOppositeDirection':
            # If the road is in the opposite direction, flows are from the ending point to the starting point
            graph.add_edge(to_node, from_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2,
                           recovery_1=recovery_1 / 2, impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                           toid=row['toid'], classification=row['classification'], geometry=row['geometry'])
        elif direction == 'inDirection':
            # If the road is in the same direction, flows are from the starting point to the ending point
            graph.add_edge(from_node, to_node, baseline_1=baseline_1 / 2, strike_1=strike_1 / 2,
                           recovery_1=recovery_1 / 2, impact_flow=impact_flow / 2, recovery_flow=recovery_flow / 2,
                           toid=row['toid'], classification=row['classification'], geometry=row['geometry'])


In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize


def graph_visualization(graph, weight=None, cmap='Greens'):
    # obtain all the edges in the graph
    edges_list = list(graph.edges())

    # make sure the graph is not empty
    if edges_list:
        # obtain the start and end coordinates of the first edge
        u, v = edges_list[0]

        # obtain the edge attribute of the first edge
        edge_attr = graph[u][v]

        print(f"The first edge: ({u}, {v})")
        print("Attribute: ", edge_attr)
    else:
        print("No edge in the graph.")

    # get the node positions
    node_positions = nx.get_node_attributes(graph, 'pos')

    # get the edge weights
    edge_weight = nx.get_edge_attributes(graph, weight)

    # normalize the edge weights between 0 and 1
    weight_values = list(edge_weight.values())
    norm = Normalize(vmin=min(weight_values), vmax=max(weight_values))
    norm_weight = {edge: norm(weight) for edge, weight in edge_weight.items()}

    # create a colormap
    cmap_object = plt.get_cmap(cmap)
    mappable = ScalarMappable(norm=norm, cmap=cmap_object)
    mappable.set_array([])

    # plot the graph
    fig, ax = plt.subplots()
    nx.draw_networkx_edges(graph, pos=node_positions, edge_color='gray')
    nx.draw_networkx_edges(graph, pos=node_positions,
                           edge_color=[mappable.to_rgba(norm_weight[edge]) for edge in edges_list])

    # add a colorbar
    cbar = plt.colorbar(mappable, ax=ax, orientation='horizontal', pad=0.01)
    cbar.set_label(weight)

    plt.title(f'Graph Representation of the Road Network', size=10)
    plt.axis('off')
    plt.show()

In [ ]:
graph_visualization(graph, weight='impact_flow')
graph_visualization(graph, weight='recovery_flow')

#### Calculate the structure-based indicators for roads and system


In [ ]:
# calculate the number of nodes
nodes_number = graph.number_of_nodes()

# calculate the number of edges
links_number = graph.number_of_edges()

# calculate the total link weight
total_link_weight = sum([data['impact_flow'] for u, v, data in graph.edges(data=True)])

mean_link_weight = total_link_weight / links_number

# calculate the coefficient of variation of node degree
node_degrees = dict(graph.degree())
mean_node_degree = np.mean(list(node_degrees.values()))
std_node_degree = np.std(list(node_degrees.values()))
node_degree_cv = (std_node_degree / mean_node_degree) * 100

# calculate the coefficient of variation of edge weight
edge_weights = nx.get_edge_attributes(graph, 'impact_flow').values()
mean_edge_weight = np.mean(list(edge_weights))
std_edge_weight = np.std(list(edge_weights))
edge_weight_cv = (std_edge_weight / mean_edge_weight) * 100

# calculate the network connectivity and score for graph
network_connectivity = nx.is_connected(graph)
connectivity_score = 2 * links_number / (nodes_number * nodes_number)

# calculate the average clustering coefficient for graph
avg_clustering_coefficient = nx.average_clustering(graph)

# calculate the transitivity for graph
transitivity = nx.transitivity(graph)

# calculate the assortativity for graph
assortativity = nx.degree_assortativity_coefficient(graph)

# calculate indicators as attributes for each road
clustering_coefficients = nx.clustering(graph)
eigenvector_centrality = nx.eigenvector_centrality(graph)

# calculate the node degrees
node_degrees = graph.degree()

# calculate the average degree for graph
average_degree = sum(dict(node_degrees).values()) / len(node_degrees)

# calculate the betweenness_centrality for each road
betweenness_centrality = nx.betweenness_centrality(graph)

# Add the calculated indicators as attributes for each road
for u, v in graph.edges():
    edge_clustering_coefficient = (clustering_coefficients[u] + clustering_coefficients[v]) / 2
    graph[u][v]['clustering_coefficient'] = edge_clustering_coefficient

    graph[u][v]['degree'] = (node_degrees[u] + node_degrees[v]) / 2
    graph[u][v]['betweenness'] = (betweenness_centrality[u] + betweenness_centrality[v]) / 2

    edge_eigenvector_centrality = (eigenvector_centrality[u] + eigenvector_centrality[v]) / 2
    graph[u][v]['eigenvector_centrality'] = edge_eigenvector_centrality

# Print the calculated indicators
print("Total Nodes Number:", nodes_number)
print("Total Links Number:", links_number)
print("Total Flows:", round(total_link_weight))
print("Mean Link Flow:", mean_link_weight)
print("Node Degree Coefficient of Variation:", node_degree_cv)
print("Edge Weight Coefficient of Variation:", edge_weight_cv)
print("Connectivity Score:", connectivity_score)
print("Network Connectivity:", network_connectivity)
print("Transitivity:", transitivity)
print("Assortativity:", assortativity)
print("Average Clustering Coefficient:", avg_clustering_coefficient)
print("Average Degree:", average_degree)
print("Average Betweenness Centrality:", sum(betweenness_centrality.values()) / len(betweenness_centrality))


impact_flow复原性和脆弱性分析：

**复原性分析：**
- **Total Nodes Number: 1104** 和 **Total Links Number: 1227** 表示路网的规模。规模较大可能需要更多资源来维护，但在复原性方面，如果网络中有多个节点和连接，也可能有更多的备选路径。

- **Total Flows: -36378** 和 **Mean Link Flow: -29.65** 表示流量变化差值和平均链路流量。负数的流量变化差值可能表示流量减少。这可能会导致某些连接拥塞或资源分配不当，对于复原性可能带来一些挑战。

- **Node Degree Coefficient of Variation: 43.65** 和 **Edge Weight Coefficient of Variation: -449.42** 表示节点度数和边权重的变异程度。负的边权重变异系数可能反映了权重值的分布不均匀。较高的节点度数变异系数可能意味着一些节点的连接更为密集，但是这种变异可能影响网络的复原性。

- **Transitivity: 0.064** 表示节点的传递性。传递性的增加可能有助于信息或流量在网络中的快速传播，提高复原性。

- **Assortativity: 0.032** 表示网络的度同配性。这种度同配性可能有助于信息在网络中传播，但是取值较低，作用可能不太明显。

**脆弱性分析：**
- **Connectivity Score: 0.00201** 和 **Network Connectivity: False** 表示连接性分数较低和网络不是全连通的。这可能意味着网络中可能存在一些孤立的区域，当这些区域受到影响时，整个网络的连通性会受到影响。

- **Average Clustering Coefficient: 0.036** 表示平均聚类系数。较低的聚类系数可能意味着节点之间关联性不高，可能导致信息难以在网络中传播。

- **Average Betweenness Centrality: 0.014** 表示平均介数中心性，较低的介数中心性可能表示网络中的信息流和影响受限。

综合分析，以流量变化差值为权重的城市伦敦路网结构可能具有一定的复原性挑战和脆弱性。负数的流量变化可能表明某些区域可能存在拥塞或资源分配不足。节点度数和边权重的变异可能意味着网络中存在一些密集连接的节点和不均匀的权重分布，这可能会影响网络的鲁棒性。然而，一些正向因素如传递性、度同配性等可能有助于信息传播和一些程度的复原。在增强路网的复原性和脆弱性方面，可能需要考虑网络的连通性、资源分配、信息传播等因素。

**Strike_1复原性分析：**
- **Total Nodes Number: 1104** 和 **Total Links Number: 1227** 表示路网的规模。路网的大小可能对于复原性有影响，大规模路网可能需要更多的资源来维持。

- **Node Degree Coefficient of Variation: 43.65** 和 **Edge Weight Coefficient of Variation: 115.01** 表示节点度数和边权重的变异程度。较高的变异系数可能意味着部分节点度数和边权重变化较大，这可能增加了复原过程的难度，因为某些节点或边更容易受到影响。

- **Transitivity: 0.064** 表示节点的传递性。传递性的增加可能有助于信息或流量在网络中的快速传播，这对复原性可能有积极影响。

- **Assortativity: 0.032** 表示网络的度同配性，即高度连接的节点倾向于连接到其他高度连接的节点。这种连接模式可能有助于信息在网络中传播，从而提高复原性。

**脆弱性分析：**
- **Network Connectivity: False** 表示网络不是全连通的，可能存在多个网络组件。这可能导致某些区域或网络组件受到故障影响时，整个网络的连通性受到影响。

- **Connectivity Score: 0.00201** 表示网络的连接性分数较低。低连接性分数可能意味着网络中的连接较弱，复原能力较弱。

- **Average Clustering Coefficient: 0.036** 表示平均聚类系数。较低的聚类系数可能表示节点之间的关联性不高，网络的鲁棒性可能较弱。

- **Average Betweenness Centrality: 0.014** 表示平均介数中心性，介数中心性衡量节点在网络中作为桥梁的程度。较低的介数中心性可能意味着网络中的信息流和影响可能受限。

综合分析，CoL路网结构可能具有一定的复原性挑战和脆弱性。较高的节点度数和边权重变异系数、低连接性分数、不高的平均聚类系数和介数中心性可能意味着在面对故障、拥塞、攻击等情况时，网络的复原能力可能受到一定影响。但是，度同配性、传递性等正向因素可能有助于一些程度上的复原和信息传播。为了增强路网的复原性和脆弱性，可能需要关注网络连接性、鲁棒性、信息传播能力等方面的改进。

In [ ]:
graph_visualization(graph, weight='betweenness')
graph_visualization(graph, weight='clustering_coefficient')
graph_visualization(graph, weight='eigenvector_centrality')

### Network DBSCAN Clustering

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# 将边的权重属性值作为数据点
edges_list = list(graph.edges())
data_points = np.array([graph[u][v]['impact_flow'] for u, v in edges_list]).reshape(-1, 1)
# 创建模型
dbscan = DBSCAN()

# 定义要搜索的参数范围
param_grid = {'eps': [1, 1.5, 2, 2.5, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples': [1, 2, 3, 4, 5, 6]}

best_score = -1
best_eps = None
best_min_samples = None

# 在数据上执行交叉验证
for eps in param_grid['eps']:
    for min_samples in param_grid['min_samples']:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(data_points)
        if len(set(labels)) > 1:  # 忽略只有一个簇的情况
            score = silhouette_score(data_points, labels)
            if score > best_score:
                best_score = score
                best_eps = eps
                best_min_samples = min_samples

print("Best EPS:", best_eps)
print("Best Min Samples:", best_min_samples)

# 使用最佳参数进行DBSCAN聚类
best_dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
best_labels = best_dbscan.fit_predict(data_points)

# 将聚类结果应用于图
for i, (u, v) in enumerate(edges_list):
    graph[u][v]['cluster_DB'] = best_labels[i]

# 打印聚类的唯一值
unique_clusters = set(best_labels)
print("Unique Cluster Values:", unique_clusters)



In [ ]:
# 创建颜色映射
cmap = plt.get_cmap('tab20', len(unique_clusters))

# 绘制图形
fig, ax = plt.subplots()
pos_cluster = nx.fruchterman_reingold_layout(graph)

for u, v, attr in graph.edges(data=True):
    cluster = attr['cluster_DB']
    if cluster == -1:
        edge_color = 'lightgrey'  # 将 -1 标签的边设置为浅灰色
    else:
        edge_color = cmap(cluster)
    nx.draw_networkx_edges(graph, pos_cluster, edgelist=[(u, v)], width=1, edge_color=edge_color)

# 创建不连续的分类点图例
unique_labels = np.unique(best_labels)
handles = []
for label in unique_labels:
    if label == -1:  # 处理 -1 标签
        handle = plt.Line2D([], [], color='lightgrey', marker='o', markersize=10, label='Noise')
    else:
        color = cmap(label)
        handle = plt.Line2D([], [], color=color, marker='o', markersize=10, label=f'Cluster {label}')
    handles.append(handle)

# 添加图例
ax.legend(handles=handles, title='Clusters', loc='upper left', bbox_to_anchor=(1, 1))

plt.axis('off')
plt.title("DBSCAN Clustering of Graph Edges")
plt.show()

node_positions = nx.get_node_attributes(graph, 'pos')
nx.draw_networkx_edges(graph, node_positions, width=1, edge_color=cmap(best_labels))
plt.title("DBSCAN Clustering in network")
plt.axis('off')
plt.show()


### Network K-Means Clustering


#### By Degree

In [ ]:
from sklearn.cluster import KMeans

# 从图中获取节点的度作为特征
node_features = np.array([graph.degree(node) for node in graph.nodes()]).reshape(-1, 1)

# 使用K均值算法进行聚类
k = 7  # 聚类数量
kmeans = KMeans(n_clusters=k)
cluster_labels = kmeans.fit_predict(node_features)

# 将聚类结果应用于图
for i, node in enumerate(graph.nodes()):
    graph.nodes[node]['cluster_k'] = cluster_labels[i]

# 初始化节点的累计flow和度的字典
node_weight = {node: 0 for node in graph.nodes()}
node_degrees = {node: 0 for node in graph.nodes()}

# 遍历边，累计flow和度
for u, v, attr in graph.edges(data=True):
    weight = attr['impact_flow']
    node_weight[u] += weight
    node_weight[v] += weight
    node_degrees[u] += 1
    node_degrees[v] += 1

# 计算每个节点的特征（平均flow）
node_features = np.array(
    [node_weight[node] / node_degrees[node] if node_degrees[node] > 0 else 0 for node in graph.nodes()]).reshape(-1, 1)

# 创建颜色映射
cmap = plt.get_cmap('tab20', k)

# 绘制图形
fig, ax = plt.subplots()
pos = nx.fruchterman_reingold_layout(graph)

for u, v, attr in graph.edges(data=True):
    u_cluster = graph.nodes[u]['cluster_k']
    v_cluster = graph.nodes[v]['cluster_k']

    if u_cluster == v_cluster:
        cluster = u_cluster
    else:
        cluster = -1  # 表示不同的聚类
    if cluster == -1:
        edge_color = 'lightgrey'  # 将不同聚类的边设置为浅灰色
    else:
        edge_color = cmap(cluster)
    nx.draw_networkx_edges(graph, pos_cluster, edgelist=[(u, v)], width=1, edge_color=edge_color)

# 创建不连续的分类点图例
unique_labels = np.unique(cluster_labels)
handles = []
for label in unique_labels:
    if label == -1:  # 处理 -1 标签
        handle = plt.Line2D([], [], color='lightgrey', marker='o', markersize=10, label='Noise')
    else:
        color = cmap(label)
        handle = plt.Line2D([], [], color=color, marker='o', markersize=10, label=f'Cluster {label}')
    handles.append(handle)

# 添加图例
ax.legend(handles=handles, title='Clusters', loc='upper left', bbox_to_anchor=(1, 1))

plt.axis('off')
plt.title("KMeans Clustering of Graph Nodes")
plt.show()

# Plot the clusters in the network
node_positions = nx.get_node_attributes(graph, 'pos')
nx.draw_networkx_edges(graph, node_positions, width=1, edge_color=cmap(cluster_labels))
# 创建不连续的分类点图例
unique_labels = np.unique(cluster_labels)
handles = []
for label in unique_labels:
    if label == -1:  # 处理 -1 标签
        handle = plt.Line2D([], [], color='lightgrey', marker='o', markersize=10, label='Noise')
    else:
        color = cmap(label)
        handle = plt.Line2D([], [], color=color, marker='o', markersize=10, label=f'Cluster {label}')
    handles.append(handle)

# 添加图例
ax.legend(handles=handles, title='Clusters', loc='upper left', bbox_to_anchor=(1, 1))
plt.title("KMeans Clustering in network")
plt.axis('off')
plt.show()

#### By Flow changes / Clustering Coefficient

In [ ]:
# 获取所有边的列表
edges_list = list(graph.edges())

# 按照flows聚类
# 初始化特征矩阵
num_edges = len(edges_list)
feature_matrix = np.zeros((num_edges, 1))  # 1列，代表 "flow" 属性值

# 填充特征矩阵
for i, (u, v) in enumerate(edges_list):
    flow = graph[u][v]['impact_flow']
    feature_matrix[i, 0] = flow

# #按照Clustering Coefficient聚类
# # 初始化特征矩阵
# num_edges = len(edges_list)
# num_nodes = len(graph.nodes())
# feature_matrix = np.zeros((num_edges, num_nodes))
# 
# # 填充特征矩阵
# for i, (u, v) in enumerate(edges_list):
#     u_idx = list(graph.nodes()).index(u)  # 获取节点 u 的整数索引
#     v_idx = list(graph.nodes()).index(v)  # 获取节点 v 的整数索引
#     u_clustering = nx.clustering(graph)[u]
#     v_clustering = nx.clustering(graph)[v]
#     feature_matrix[i, u_idx] = u_clustering
#     feature_matrix[i, v_idx] = v_clustering

# 使用K均值算法进行聚类
k = 5  # 聚类数量
kmeans = KMeans(n_clusters=k)
cluster_labels = kmeans.fit_predict(feature_matrix)

# 将聚类结果应用于图
for i, (u, v) in enumerate(edges_list):
    graph[u][v]['cluster_k'] = cluster_labels[i]

# 创建颜色映射
cmap = plt.get_cmap('tab20', k)

# 绘制图形
fig, ax = plt.subplots()
pos = nx.spring_layout(graph)

for u, v, attr in graph.edges(data=True):
    cluster = attr['cluster_k']
    edge_color = cmap(cluster)
    nx.draw_networkx_edges(graph, pos, edgelist=[(u, v)], width=1, edge_color=edge_color)

# 创建不连续的分类点图例
unique_labels = np.unique(cluster_labels)
handles = []
for label in unique_labels:
    color = cmap(label)
    handle = plt.Line2D([], [], color=color, marker='o', markersize=10, label=f'Cluster {label}')
    handles.append(handle)

ax.legend(handles=handles, title='Clusters', loc='upper left', bbox_to_anchor=(1, 1))
plt.axis('off')
plt.title("Edge Flow Changes-based Clustering")
plt.show()

# Plot the clusters in the network
node_positions = nx.get_node_attributes(graph, 'pos')
nx.draw_networkx_edges(graph, node_positions, width=1, edge_color=cmap(cluster_labels))
# 创建不连续的分类点图例
unique_labels = np.unique(cluster_labels)
handles = []
for label in unique_labels:
    if label == -1:  # 处理 -1 标签
        handle = plt.Line2D([], [], color='lightgrey', marker='o', markersize=10, label='Noise')
    else:
        color = cmap(label)
        handle = plt.Line2D([], [], color=color, marker='o', markersize=10, label=f'Cluster {label}')
    handles.append(handle)

# 添加图例
ax.legend(handles=handles, title='Clusters', loc='upper left', bbox_to_anchor=(1, 1))
plt.title("KMeans Clustering in network")
plt.axis('off')
plt.show()



### Export the graph and update the indicators to All dataframe

In [ ]:
update = All.copy()

# 创建一个空的 Pandas DataFrame
columns = ['toid', 'clustering_coefficient', 'degree', 'betweenness',
           'eigenvector_centrality', 'cluster_DB', 'cluster_k']
data = []

# 将图的每条边导出到 DataFrame
for _, _, edge_data in graph.edges(data=True):
    row_data = {'toid': edge_data['toid'],
                # 'strike_1': edge_data['strike_1'], 'baseline_1': edge_data['baseline_1'],
                # 'recovery_1': edge_data['recovery_1'],
                # 'impact_flow': edge_data['impact_flow'], 'recovery_flow': edge_data['recovery_flow'],
                # 'classification': edge_data['classification'], 'geometry': edge_data['geometry'],
                'clustering_coefficient': edge_data['clustering_coefficient'], 'degree': edge_data['degree'],
                'betweenness': edge_data['betweenness'], 'eigenvector_centrality': edge_data['eigenvector_centrality'],
                'cluster_DB': edge_data['cluster_DB'], 'cluster_k': edge_data['cluster_k']}
    data.append(row_data)

# 创建 DataFrame
graph_df = pd.DataFrame(data, columns=columns)
graph_df['mode']='total'

update['toid'] = update['toid'].astype(str)

update = pd.merge(update, graph_df, on=['toid','mode'], how='left')




In [ ]:
# need to merge all updates to All dataframe
update

## Flow changes

In [ ]:
import plotly.graph_objects as go

# 示例数据
source = [0, 0, 1, 1, 2]
target = [2, 3, 3, 4, 4]
value = [120, 20, 50, 30, 40]
labels = ['Input 1', 'Input 2', 'Output 1', 'Output 2', 'Output 3']

# 创建Sankey图
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
))

fig.update_layout(title_text="Sankey Diagram", font_size=10)
fig.show()
